In [184]:
!pip install matplotlib
!pip install seaborn


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Loading Data

In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# set file paths
imdb_path = r"../datasets/title.basics.tsv.gz"
ratings_path = r"../datasets/title.ratings.tsv.gz"
principals_path = r"../datasets/title.principals.tsv.gz"
name_path = r"../datasets/name.basics.tsv.gz"
tmdb_path = r"../datasets/TMDB_movie_dataset_v11.csv"

# Load IMDb datasets
title_basics = pd.read_csv(
    imdb_path, 
    sep="\t", 
    low_memory=False, 
    usecols=["tconst", "titleType", "primaryTitle", "isAdult", "startYear", "runtimeMinutes", "genres"],
    na_values="\\N"
)

title_ratings = pd.read_csv(
    ratings_path, 
    sep="\t", 
    low_memory=False, 
    na_values="\\N"
)

title_principals = pd.read_csv(
    principals_path, 
    sep="\t", 
    low_memory=False, 
    usecols=["tconst", "nconst", "category"],
    na_values="\\N"
)

name_basics = pd.read_csv(
    name_path, 
    sep="\t", 
    low_memory=False, 
    usecols=["nconst", "primaryName", "primaryProfession"],
    na_values="\\N"
)

#Load TMDb dataset
tmdb_movies = pd.read_csv(
    tmdb_path, 
    low_memory=False,
    usecols=["imdb_id", "title", "vote_average", "vote_count", "release_date", "revenue", "runtime", "budget", "overview", "popularity", "tagline", "genres", "production_companies", "keywords"]
)
tmdb_movies = tmdb_movies.rename(columns={"genres": "genres_tmdb"})

print(title_basics.head())
print(title_ratings.head())
print(title_principals.head())
print(name_basics.head())
print(tmdb_movies.head())

      tconst titleType            primaryTitle  isAdult  startYear  \
0  tt0000001     short              Carmencita      0.0     1894.0   
1  tt0000002     short  Le clown et ses chiens      0.0     1892.0   
2  tt0000003     short            Poor Pierrot      0.0     1892.0   
3  tt0000004     short             Un bon bock      0.0     1892.0   
4  tt0000005     short        Blacksmith Scene      0.0     1893.0   

  runtimeMinutes                    genres  
0              1         Documentary,Short  
1              5           Animation,Short  
2              5  Animation,Comedy,Romance  
3             12           Animation,Short  
4              1                     Short  
      tconst  averageRating  numVotes
0  tt0000001            5.7      2125
1  tt0000002            5.6       286
2  tt0000003            6.4      2161
3  tt0000004            5.3       183
4  tt0000005            6.2      2890
      tconst     nconst         category
0  tt0000001  nm1588970             self

In [186]:
title_basics = title_basics[title_basics['titleType'] == 'movie']
tmdb_movies.rename(columns={"imdb_id": "tconst"}, inplace=True)
merged_movies = title_basics.merge(tmdb_movies, on="tconst", how="inner")

merged_movies = merged_movies.drop_duplicates()

row_count_merged = len(merged_movies)
print(row_count_merged)

row_count_title_basics = len(title_basics)
print(row_count_title_basics)

335317
704881


Join IMDb parts to create Master IMDb set 

In [187]:
print(merged_movies.columns.tolist())

['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear', 'runtimeMinutes', 'genres', 'title', 'vote_average', 'vote_count', 'release_date', 'revenue', 'runtime', 'budget', 'overview', 'popularity', 'tagline', 'genres_tmdb', 'production_companies', 'keywords']


In [188]:
imdb_df = title_basics.merge(title_ratings, on="tconst", how="left")
imdb_df = imdb_df.merge(title_principals, on="tconst", how="left")
imdb_df = imdb_df.merge(name_basics, on="nconst", how="left")

print(imdb_df.head())

      tconst titleType primaryTitle  isAdult  startYear runtimeMinutes  \
0  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
1  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
2  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
3  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
4  tt0000009     movie   Miss Jerry      0.0     1894.0             45   

    genres  averageRating  numVotes     nconst  category        primaryName  \
0  Romance            5.4     218.0  nm0063086   actress    Blanche Bayliss   
1  Romance            5.4     218.0  nm0183823     actor  William Courtenay   
2  Romance            5.4     218.0  nm1309758     actor     Chauncey Depew   
3  Romance            5.4     218.0  nm0085156  director    Alexander Black   
4  Romance            5.4     218.0  nm0085156    writer    Alexander Black   

                 primaryProfession  
0                          actress  
1     

Merge IMDb and TMDb and filter only for "movie" to create the overall master movie dataset

In [189]:
tmdb_movies.rename(columns={"imdb_id": "tconst"}, inplace=True)

# inner join IMDb and TMDb datasets
merged_df = imdb_df.merge(tmdb_movies, on="tconst", how="inner")

# filter only where titleType = movie
merged_df = merged_df[merged_df['titleType'] == 'movie']

print(merged_df.head())

      tconst titleType primaryTitle  isAdult  startYear runtimeMinutes  \
0  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
1  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
2  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
3  tt0000009     movie   Miss Jerry      0.0     1894.0             45   
4  tt0000009     movie   Miss Jerry      0.0     1894.0             45   

    genres  averageRating  numVotes     nconst  ... release_date revenue  \
0  Romance            5.4     218.0  nm0063086  ...   1894-10-08       0   
1  Romance            5.4     218.0  nm0183823  ...   1894-10-08       0   
2  Romance            5.4     218.0  nm1309758  ...   1894-10-08       0   
3  Romance            5.4     218.0  nm0085156  ...   1894-10-08       0   
4  Romance            5.4     218.0  nm0085156  ...   1894-10-08       0   

  runtime budget                                           overview  \
0      45      0  The adven

In [190]:
print(merged_df.columns.tolist())

['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'nconst', 'category', 'primaryName', 'primaryProfession', 'title', 'vote_average', 'vote_count', 'release_date', 'revenue', 'runtime', 'budget', 'overview', 'popularity', 'tagline', 'genres_tmdb', 'production_companies', 'keywords']


Specific Table Separation

In [191]:
# Create Movies Only Table
movies_df = merged_df[['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'title', 'vote_average', 'vote_count', 'release_date', 'revenue', 'runtime', 'budget', 'overview', 'popularity', 'tagline', 'genres_tmdb', 'production_companies', 'keywords']]
movies_df = movies_df.drop_duplicates()
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])
movies_df['release_month'] = movies_df['release_date'].dt.month

## create consolidated runtime column:
movies_df['runtimeMinutes'] = pd.to_numeric(movies_df['runtimeMinutes'], errors='coerce')
movies_df['runtime'] = pd.to_numeric(movies_df['runtime'], errors='coerce')

conditions = [
    (movies_df['runtimeMinutes'] == movies_df['runtime']),
    (movies_df['runtimeMinutes'] > movies_df['runtime']),
    (movies_df['runtime'] > movies_df['runtimeMinutes']),
    (movies_df['runtimeMinutes'].isna() & movies_df['runtime'].notna()),
    (movies_df['runtime'].isna() & movies_df['runtimeMinutes'].notna()),
]

choices = [
    movies_df['runtime'], # equal runtimes
    movies_df['runtimeMinutes'], # imdb runtime longer
    movies_df['runtime'], # tmdb runtime longer
    movies_df['runtime'], # imdb NA, use tmdb
    movies_df['runtimeMinutes'] # tmdb NA, use imdb
]

movies_df['runtime_consolidated'] = np.select(conditions, choices, default=np.nan)

## Create Actors Table, Actresses Table, and Directors Table
cast_and_crew_df = merged_df[['tconst', 'nconst', 'primaryName', 'category']]
cast_and_crew_df = cast_and_crew_df.drop_duplicates(subset=['tconst', 'nconst'])

# Actors table 
actor_df = cast_and_crew_df[cast_and_crew_df['category'] == 'actor']
# Actress table
actress_df = cast_and_crew_df[cast_and_crew_df['category'] == 'actress']
# Directors table
director_df = cast_and_crew_df[cast_and_crew_df['category'] == 'director']

row_number = len(movies_df)
print(row_number)

print(movies_df.head())
print(actor_df.head())
print(actress_df.head())
print(director_df.head())


335317
       tconst titleType                   primaryTitle  isAdult  startYear  \
0   tt0000009     movie                     Miss Jerry      0.0     1894.0   
7   tt0000147     movie  The Corbett-Fitzsimmons Fight      0.0     1897.0   
15  tt0000574     movie    The Story of the Kelly Gang      0.0     1906.0   
35  tt0000591     movie               The Prodigal Son      0.0     1907.0   
41  tt0000615     movie             Robbery Under Arms      0.0     1907.0   

    runtimeMinutes                      genres  averageRating  numVotes  \
0             45.0                     Romance            5.4     218.0   
7            100.0      Documentary,News,Sport            5.3     548.0   
15            70.0  Action,Adventure,Biography            6.0     969.0   
35            90.0                       Drama            5.6      30.0   
41             NaN                       Drama            4.3      27.0   

                            title  ...  runtime  budget  \
0             

Table Explosions (for detailed analysis)

In [192]:
# split and explode genres (imdb)
genres_df = merged_df[['tconst', 'genres']].dropna(subset=['genres'])
genres_df['genres'] = genres_df['genres'].str.split(',')
genres_df = genres_df.explode('genres')

# split and explode production companies
prod_comp_df = merged_df[['tconst', 'production_companies']].dropna(subset=['production_companies'])
prod_comp_df['production_companies'] = prod_comp_df['production_companies'].str.split(',')
prod_comp_df = prod_comp_df.explode('production_companies')

# split and explode keywords
keyword_df = merged_df[['tconst', 'keywords']].dropna(subset=['keywords'])
keyword_df['keywords'] = keyword_df['keywords'].str.split(',')
keyword_df = keyword_df.explode('keywords')

genres_df = genres_df.drop_duplicates()
prod_comp_df = prod_comp_df.drop_duplicates()
keyword_df = keyword_df.drop_duplicates()

print(genres_df.head())
print(prod_comp_df.head())
print(keyword_df.head())

       tconst       genres
0   tt0000009      Romance
7   tt0000147  Documentary
7   tt0000147         News
7   tt0000147        Sport
15  tt0000574       Action
       tconst        production_companies
0   tt0000009  Alexander Black Photoplays
7   tt0000147           Veriscope Company
15  tt0000574          Johnson and Gibson
15  tt0000574                J. & N. Tait
35  tt0000591                Pathé Frères
       tconst     keywords
0   tt0000009     reporter
0   tt0000009       editor
0   tt0000009    lost film
7   tt0000147  silent film
15  tt0000574  silent film


Cleanup time

In [193]:
# handle missing values -- removal of rows without startyear, runtimeMinutes, averageRating, numVotes, 
movies_df = movies_df.dropna(subset=['startYear', 'averageRating', 'numVotes', 'release_date'])
movies_df['overview'] = movies_df['overview'].fillna('NaN')
movies_df['tagline'] = movies_df['tagline'].fillna('NaN')
movies_df['genres'] = movies_df['genres'].fillna('NaN')
movies_df['genres_tmdb'] = movies_df['genres_tmdb'].fillna('NaN')
movies_df['production_companies'] = movies_df['production_companies'].fillna('NaN')
movies_df['keywords'] = movies_df['keywords'].fillna('NaN')

# remove duplicates
movies_df = movies_df.drop_duplicates()

# rearrange columns 
movies_df = movies_df[['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear', 'release_month', 'release_date', 'runtime_consolidated', 'averageRating', 'numVotes', 'vote_average', 'vote_count', 'revenue', 'budget', 'popularity', 'overview', 'tagline', 'genres', 'production_companies', 'keywords']]
movies_df = movies_df.rename(columns={
    'startYear': 'releaseYear', 
    'release_date': 'releaseDate', 
    'release_month': 'releaseMonth', 
    'averageRating': 'imdb_averageRating', 
    'numVotes': 'imdb_numVotes', 
    'vote_average': 'tmdb_averageRating', 
    'vote_count': 'tmdb_numVotes'
})

# convert data types
movies_df['releaseYear'] = movies_df['releaseYear'].astype('Int64')
movies_df['releaseMonth'] = movies_df['releaseMonth'].astype('Int64')


# check missing values 
row_count = len(movies_df)

print(row_count)
print(movies_df.isnull().sum())
print(movies_df.head())



260135
tconst                  0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
overview                0
tagline                 0
genres                  0
production_companies    0
keywords                0
dtype: int64
       tconst titleType                   primaryTitle  isAdult  releaseYear  \
0   tt0000009     movie                     Miss Jerry      0.0         1894   
7   tt0000147     movie  The Corbett-Fitzsimmons Fight      0.0         1897   
15  tt0000574     movie    The Story of the Kelly Gang      0.0         1906   
35  tt0000591     movie               The Prodigal Son      0.0         1907   
41  tt0000615     movie             Robbery Under Arms      

In [194]:
actor_df = actor_df.dropna(subset='primaryName')
actor_df = actor_df.merge(movies_df, on='tconst', how='inner')
actor_df = actor_df[['tconst', 'nconst', 'primaryName', 'category', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]


actor_df_rn = len(actor_df)

print(actor_df_rn) 
print(actor_df.isnull().sum())
print(actor_df.head())

1349057
tconst                  0
nconst                  0
primaryName             0
category                0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst     nconst        primaryName category titleType  \
0  tt0000009  nm0183823  William Courtenay    actor     movie   
1  tt0000009  nm1309758     Chauncey Depew    actor     movie   
2  tt0000574  nm0846894          John Tait    actor     movie   
3  tt0000574  nm1431224  Nicholas Brierley    actor     movie   
4  tt0000574  nm3002376    Norman Campbell    actor     movie   

                  primaryTitle  isAdult  releaseYear  releaseMonth  \
0                   Miss Jerry      0.0         18

In [195]:

actress_df = actress_df.merge(movies_df, on='tconst', how='inner')
actress_df = actress_df[['tconst', 'nconst', 'primaryName', 'category', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]

actress_df_rn = len(actress_df)

print(actress_df_rn)
print(actress_df.isnull().sum())
print(actress_df.head())

743679
tconst                  0
nconst                  0
primaryName             0
category                0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst     nconst          primaryName category titleType  \
0  tt0000009  nm0063086      Blanche Bayliss  actress     movie   
1  tt0000574  nm0846887       Elizabeth Tait  actress     movie   
2  tt0000574  nm0170118           Bella Cola  actress     movie   
3  tt0000591  nm1323543  Christiane Mandelys  actress     movie   
4  tt0000591  nm1759558       Gilberte Sergy  actress     movie   

                  primaryTitle  isAdult  releaseYear  releaseMonth  \
0                   Miss Jerry      0.0

In [196]:
director_df = director_df.merge(movies_df, on='tconst', how='inner')
director_df = director_df[['tconst', 'nconst', 'primaryName', 'category', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]

director_df_rn = len(director_df)

print(director_df_rn)
print(director_df.isnull().sum())
print(director_df.head())

258186
tconst                  0
nconst                  0
primaryName             0
category                0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst     nconst       primaryName  category titleType  \
0  tt0000009  nm0085156   Alexander Black  director     movie   
1  tt0000147  nm0714557   Enoch J. Rector  director     movie   
2  tt0000574  nm0846879      Charles Tait  director     movie   
3  tt0000591  nm0141150      Michel Carré  director     movie   
4  tt0000615  nm0533958  Charles MacMahon  director     movie   

                    primaryTitle  isAdult  releaseYear  releaseMonth  \
0                     Miss Jerry      0.0        

In [197]:
# combine cast & crew dfs

actor_actress_director_df = pd.concat([actor_df, actress_df, director_df], ignore_index=True)

actor_actress_director_df_rn = len(actor_actress_director_df)

print(actor_actress_director_df_rn)
print(actor_actress_director_df.isnull().sum())
print(actor_actress_director_df.head())

2350922
tconst                  0
nconst                  0
primaryName             0
category                0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst     nconst        primaryName category titleType  \
0  tt0000009  nm0183823  William Courtenay    actor     movie   
1  tt0000009  nm1309758     Chauncey Depew    actor     movie   
2  tt0000574  nm0846894          John Tait    actor     movie   
3  tt0000574  nm1431224  Nicholas Brierley    actor     movie   
4  tt0000574  nm3002376    Norman Campbell    actor     movie   

                  primaryTitle  isAdult  releaseYear  releaseMonth  \
0                   Miss Jerry      0.0         18

In [198]:
print(genres_df.columns.tolist())
print(movies_df.columns.tolist())

['tconst', 'genres']
['tconst', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity', 'overview', 'tagline', 'genres', 'production_companies', 'keywords']


In [199]:
movie_stats = movies_df[['tconst', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating' ,'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]
genres_df = genres_df.merge(movie_stats, on='tconst', how='inner')
genres_df = genres_df[['tconst', 'genres', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]

genres_df_rn = len(genres_df)

print(genres_df_rn)
print(genres_df.isnull().sum())
print(genres_df.head())

457841
tconst                  0
genres                  0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst       genres titleType                   primaryTitle  isAdult  \
0  tt0000009      Romance     movie                     Miss Jerry      0.0   
1  tt0000147  Documentary     movie  The Corbett-Fitzsimmons Fight      0.0   
2  tt0000147         News     movie  The Corbett-Fitzsimmons Fight      0.0   
3  tt0000147        Sport     movie  The Corbett-Fitzsimmons Fight      0.0   
4  tt0000574       Action     movie    The Story of the Kelly Gang      0.0   

   releaseYear  releaseMonth releaseDate  runtime_consolidated  \
0         1894         

In [200]:
prod_comp_df = prod_comp_df.merge(movie_stats, on='tconst', how='inner')
prod_comp_df = prod_comp_df[['tconst', 'production_companies', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]

prod_comp_df_rn = len(prod_comp_df)

print(prod_comp_df_rn)
print(prod_comp_df.isnull().sum())
print(prod_comp_df.head())

312726
tconst                  0
production_companies    0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst        production_companies titleType  \
0  tt0000009  Alexander Black Photoplays     movie   
1  tt0000147           Veriscope Company     movie   
2  tt0000574          Johnson and Gibson     movie   
3  tt0000574                J. & N. Tait     movie   
4  tt0000591                Pathé Frères     movie   

                    primaryTitle  isAdult  releaseYear  releaseMonth  \
0                     Miss Jerry      0.0         1894            10   
1  The Corbett-Fitzsimmons Fight      0.0         1897             1   
2    The Story of the K

In [201]:
print(keyword_df.columns.tolist())

['tconst', 'keywords']


In [202]:
keyword_df = keyword_df.merge(movie_stats, on='tconst', how='inner')
keyword_df = keyword_df[['tconst', 'keywords', 'titleType', 'primaryTitle', 'isAdult', 'releaseYear', 'releaseMonth', 'releaseDate', 'runtime_consolidated', 'imdb_averageRating', 'imdb_numVotes', 'tmdb_averageRating', 'tmdb_numVotes', 'revenue', 'budget', 'popularity']]

keyword_df_rn = len(keyword_df)

print(keyword_df_rn) 
print(keyword_df.isnull().sum())
print(keyword_df.head())

455615
tconst                  0
keywords                0
titleType               0
primaryTitle            0
isAdult                 0
releaseYear             0
releaseMonth            0
releaseDate             0
runtime_consolidated    0
imdb_averageRating      0
imdb_numVotes           0
tmdb_averageRating      0
tmdb_numVotes           0
revenue                 0
budget                  0
popularity              0
dtype: int64
      tconst     keywords titleType                   primaryTitle  isAdult  \
0  tt0000009     reporter     movie                     Miss Jerry      0.0   
1  tt0000009       editor     movie                     Miss Jerry      0.0   
2  tt0000009    lost film     movie                     Miss Jerry      0.0   
3  tt0000147  silent film     movie  The Corbett-Fitzsimmons Fight      0.0   
4  tt0000574  silent film     movie    The Story of the Kelly Gang      0.0   

   releaseYear  releaseMonth releaseDate  runtime_consolidated  \
0         1894         

### Final Dataframes
* movies_df
* actor_actress_director_df
* genres_df
* prod_comp_df
* keyword_df

In [203]:

movies_df['isAdult'] = movies_df['isAdult'].astype('int64')

movies_df['averageRating'] = (movies_df['imdb_averageRating'] + movies_df['tmdb_averageRating']) / 2
movies_df['averageNumVotes'] = (movies_df['imdb_numVotes'] + movies_df['tmdb_numVotes']) / 2

print(movies_df.describe())
print(movies_df.dtypes)

movies_df.to_csv('../data/movies_data.csv', index=False)

             isAdult  releaseYear  releaseMonth  \
count  260135.000000     260135.0      260135.0   
mean        0.013478  1993.801257      6.307202   
min         0.000000       1894.0           1.0   
25%         0.000000       1976.0           3.0   
50%         0.000000       2004.0           6.0   
75%         0.000000       2016.0          10.0   
max         1.000000       2025.0          12.0   
std         0.115308    26.702713       3.68064   

                         releaseDate  runtime_consolidated  \
count                         260135         260135.000000   
mean   1994-05-01 15:51:30.725969152             93.435701   
min              1894-10-08 00:00:00              0.000000   
25%              1976-08-27 12:00:00             81.000000   
50%              2004-07-16 00:00:00             92.000000   
75%              2016-02-11 00:00:00            104.000000   
max              2027-04-21 00:00:00          59460.000000   
std                              NaN        

In [204]:
actor_actress_director_df['isAdult'] = actor_actress_director_df['isAdult'].astype('int64')

actor_actress_director_df['averageRating'] = (actor_actress_director_df['imdb_averageRating'] + actor_actress_director_df['tmdb_averageRating']) / 2
actor_actress_director_df['averageNumVotes'] = (actor_actress_director_df['imdb_numVotes'] + actor_actress_director_df['tmdb_numVotes']) / 2

print(actor_actress_director_df.describe())
print(actor_actress_director_df.dtypes)

actor_actress_director_df.to_csv('../data/actor_actress_director.csv', index=False)

            isAdult  releaseYear  releaseMonth                    releaseDate  \
count  2.350922e+06    2350922.0     2350922.0                        2350922   
mean   1.365167e-02  1991.221075      6.341545  1991-10-01 18:48:50.916806400   
min    0.000000e+00       1894.0           1.0            1894-10-08 00:00:00   
25%    0.000000e+00       1972.0           3.0            1972-10-26 00:00:00   
50%    0.000000e+00       2000.0           6.0            2000-01-01 00:00:00   
75%    0.000000e+00       2015.0          10.0            2015-01-29 00:00:00   
max    1.000000e+00       2025.0          12.0            2027-04-21 00:00:00   
std    1.160401e-01    27.269342        3.6761                            NaN   

       runtime_consolidated  imdb_averageRating  imdb_numVotes  \
count          2.350922e+06        2.350922e+06   2.350922e+06   
mean           9.501584e+01        5.905442e+00   5.380360e+03   
min            0.000000e+00        1.000000e+00   5.000000e+00   
25%   

In [205]:
genres_df['isAdult'] = genres_df['isAdult'].astype('int64')

genres_df['averageRating'] = (genres_df['imdb_averageRating'] + genres_df['tmdb_averageRating']) / 2
genres_df['averageNumVotes'] = (genres_df['imdb_numVotes'] + genres_df['tmdb_numVotes']) / 2

print(genres_df.describe())
print(genres_df.dtypes)

genres_df.to_csv('../data/genres_data.csv', index=False)

             isAdult  releaseYear  releaseMonth  \
count  457841.000000     457841.0      457841.0   
mean        0.012273  1993.800027      6.371861   
min         0.000000       1894.0           1.0   
25%         0.000000       1976.0           3.0   
50%         0.000000       2004.0           6.0   
75%         0.000000       2015.0          10.0   
max         1.000000       2025.0          12.0   
std         0.110101    26.625999      3.655163   

                         releaseDate  runtime_consolidated  \
count                         457841         457841.000000   
mean   1994-05-02 17:27:00.460378240             95.096898   
min              1894-10-08 00:00:00              0.000000   
25%              1976-07-07 00:00:00             83.000000   
50%              2004-08-06 00:00:00             93.000000   
75%              2015-11-24 00:00:00            105.000000   
max              2027-04-21 00:00:00          59460.000000   
std                              NaN        

In [206]:
prod_comp_df['isAdult'] = prod_comp_df['isAdult'].astype('int64')

prod_comp_df['averageRating'] = (prod_comp_df['imdb_averageRating'] + prod_comp_df['tmdb_averageRating']) / 2
prod_comp_df['averageNumVotes'] = (prod_comp_df['imdb_numVotes'] + prod_comp_df['tmdb_numVotes']) / 2

print(prod_comp_df.describe())
print(prod_comp_df.dtypes)

prod_comp_df.to_csv('../data/production_companies_data.csv', index=False)

             isAdult  releaseYear  releaseMonth  \
count  312726.000000     312726.0      312726.0   
mean        0.008576  1996.948002      6.616565   
min         0.000000       1894.0           1.0   
25%         0.000000       1982.0           3.0   
50%         0.000000       2007.0           7.0   
75%         0.000000       2017.0          10.0   
max         1.000000       2025.0          12.0   
std         0.092210    25.388075      3.567469   

                         releaseDate  runtime_consolidated  \
count                         312726         312726.000000   
mean   1997-07-08 21:47:08.763838208             97.398640   
min              1894-10-08 00:00:00              0.000000   
25%              1982-04-22 00:00:00             86.000000   
50%              2007-08-04 00:00:00             95.000000   
75%              2017-06-25 00:00:00            107.000000   
max              2025-03-06 00:00:00          59460.000000   
std                              NaN        

In [207]:
keyword_df['isAdult'] = keyword_df['isAdult'].astype('int64')

keyword_df['averageRating'] = (keyword_df['imdb_averageRating'] + keyword_df['tmdb_averageRating']) / 2
keyword_df['averageNumVotes'] = (keyword_df['imdb_numVotes'] + keyword_df['tmdb_numVotes']) / 2

print(keyword_df.describe())
print(keyword_df.dtypes)

keyword_df.to_csv('../data/keyword_data.csv', index=False)

             isAdult  releaseYear  releaseMonth  \
count  455615.000000     455615.0      455615.0   
mean        0.009163  1993.528499      6.681453   
min         0.000000       1894.0           1.0   
25%         0.000000       1976.0           4.0   
50%         0.000000       2003.0           7.0   
75%         0.000000       2015.0          10.0   
max         1.000000       2025.0          12.0   
std         0.095286    26.484911      3.525325   

                         releaseDate  runtime_consolidated  \
count                         455615         455615.000000   
mean   1994-02-04 13:37:37.503374592             98.309527   
min              1894-10-08 00:00:00              0.000000   
25%              1976-08-12 00:00:00             85.000000   
50%              2003-12-19 00:00:00             95.000000   
75%              2015-09-22 00:00:00            108.000000   
max              2025-03-06 00:00:00           1620.000000   
std                              NaN        